In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np

import osu.dataset as dataset


osu! path: C:\Users\niooi\AppData\Roaming/../Local/osu!


In [2]:
# reload from saved

xs = np.load(f'.datasets/xs_5000_07-21_23-28-07.npy')
ys = np.load(f'.datasets/ys_5000_07-21_23-28-07.npy')

# xs = np.load(f'.datasets/xs_142_07-21_21-50-55.npy')
# ys = np.load(f'.datasets/ys_142_07-21_21-50-55.npy')

In [6]:
import torch

BATCH_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

2157

In [18]:
from osu.vae import OsuReplayVAE

vae = OsuReplayVAE(batch_size=BATCH_SIZE)

vae.load_data(xs, ys)

VAE models compiled..
VAE Models initialized on cuda
Encoder parameters: 212032
Decoder parameters: 219394
Data loaded: 14082 training samples, 3521 test samples (position only)


In [23]:

VAE_EPOCHS = 5

for i in range(1):
    vae.train(VAE_EPOCHS)
    vae.save()
    
vae.plot_losses()

Epoch 1/5 (VAE):   0%|          | 0/111 [00:00<?, ?it/s]


RuntimeError: cudnn RNN backward can only be called in training mode

In [20]:
# test training results
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset

test_name = 'ringyou'
test_mods = Mods.DOUBLE_TIME
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  4.95it/s]


In [21]:
replay_data = vae.generate(data)
    
replay_data = np.concatenate(replay_data)
replay_data = np.pad(replay_data, ((0, 0), (0, 2)), mode='constant', constant_values=0)
if not os.path.exists('.generated'):
    os.makedirs('.generated')
    
print(f"Generated replay data shape: {replay_data.shape}")


Generated replay data shape: (8192, 4)


In [22]:
import osu.preview.preview as preview

preview.preview_replay_raw(replay_data, test_map_path, test_mods, test_song)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
